In [ ]:
import os
import random
import shutil

random.seed(42)

IMG_DIR = 'data/images'
LBL_DIR = 'data/annotations'

SPLIT_DIR = 'data/split'
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(SPLIT_DIR, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(SPLIT_DIR, split, 'labels'), exist_ok=True)

files = [f for f in os.listdir(IMG_DIR) if f.endswith('.tif')]
random.shuffle(files)

n = len(files)
train_split = int(n * 0.6)
val_split = int(n * 0.8)

for i, f in enumerate(files):
    split = 'train' if i < train_split else 'val' if i < val_split else 'test'
    shutil.copy(os.path.join(IMG_DIR, f), os.path.join(SPLIT_DIR, split, 'images', f))
    label_name = f.replace('.tif', '.txt')
    shutil.copy(os.path.join(LBL_DIR, label_name), os.path.join(SPLIT_DIR, split, 'labels', label_name))

print(f"Split complete: {train_split} train, {val_split-train_split} val, {n-val_split} test")

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt

model.train(
    data='data/data.yaml',
    epochs=20,
    imgsz=1024,
    batch=4,
    project='urban_yolo_training',
    name='yolov8_finetuned',
    exist_ok=True
)

In [ ]:
import csv
from collections import Counter

TEST_DIR = 'data/split/test/images'
MODEL_PATH = 'urban_yolo_training/yolov8_finetuned/weights/best.pt'
OUTPUT_CSV = 'results/yolov8_counts_and_class.csv'

os.makedirs('results', exist_ok=True)
model = YOLO(MODEL_PATH)

def classify_zone(buildings, roads, vegetation, construction):
    if construction > vegetation:
        return 'construction_zone'
    elif vegetation > 25 and buildings < 10:
        return 'green_recovery_zone'
    elif buildings > 20 and construction < 3:
        return 'urban_growth_zone'
    else:
        return 'stable_zone'

with open(OUTPUT_CSV, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_id', 'buildings', 'roads', 'vegetation', 'construction_zones', 'zone_class'])

    for image_file in os.listdir(TEST_DIR):
        if not image_file.endswith('.tif'):
            continue

        results = model.predict(source=os.path.join(TEST_DIR, image_file), conf=0.25, save=False, verbose=False)
        class_ids = [int(box.cls) for box in results[0].boxes]

        counts = Counter(class_ids)
        b = counts.get(0, 0)
        r = counts.get(1, 0)
        v = counts.get(2, 0)
        c = counts.get(3, 0)

        zone_class = classify_zone(b, r, v, c)

        writer.writerow([image_file, b, r, v, c, zone_class])

print(f"CSV generated with zone classes: {OUTPUT_CSV}")
